# Análisis de Discurso de Odio en Español
Este cuaderno presenta un pipeline profesional de NLP para caracterizar textos con posibles patrones de discurso de odio utilizando spaCy y análisis estadístico.


In [1]:
# Limpieza inicial del dataset (encoding y caracteres raros) 
# Basado en el script compartido por el profesor en el foro
# Objetivo: dejar el corpus en UTF-8 legible

from pathlib import Path
import pandas as pd

# Rutas de entrada/salida
input_path = Path("Dataset.csv")         # fichero original 
tmp_path = Path("Dataset_tmp_sin_bytes_raros.csv")   # fichero intermedio
output_path = Path("Dataset_comentarios_limpios_utf8.csv")      # fichero final limpio

# Algunos bytes problemáticos detectados en el archivo original
# (provocan símbolos raros si no se eliminan)
BAD_BYTES = {0xBF, 0xA1, 0xB3}  

# Eliminamos esos bytes a nivel binario
with input_path.open("rb") as f_in, tmp_path.open("wb") as f_out:
    for raw_line in f_in:
        cleaned = bytes(b for b in raw_line if b not in BAD_BYTES)
        f_out.write(cleaned)

# Leemos el CSV  con codificación latin1
df = pd.read_csv(tmp_path, sep = ";", encoding = "latin1", low_memory = False)

def arreglar_mojibake(s):
    #Función de limpieza de texto: Aplica una tabla de reemplazos manual para casos que no se corrigen bien.
    if not isinstance(s, str):
        return s
    
    # Primero intento el patrón latin1 a utf-8
    try:
        reparado = s.encode("latin1").decode("utf-8")
    except UnicodeError:
        reparado = s
    
    # Después aplico los reemplazos manuales (Ã± ñ Â¿ ¿)
    for malo, bueno in REEMPLAZOS.items():
        reparado = reparado.replace(malo, bueno)
    
    return reparado

# Aplico la limpieza a todas las columnas de texto (tipo object)
text_cols = df.select_dtypes(include = "object").columns
for col in text_cols:
    df[col] = df[col].apply(arreglar_mojibake)

# Elimino columnas sin nombre que vienen vacías
df = df.loc[:, ~df.columns.str.startswith("Unnamed")]

# Guardamos el utf-8 limpio
df.to_csv(output_path, sep = ";", index=False, encoding = "utf-8")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '02Dataset_anonimizado.csv'

In [2]:
import pathlib
import spacy
import pandas as pd
from spacy import displacy
import csv
import es_core_news_md

In [3]:
nlp = es_core_news_md.load()

In [4]:
# Nombre del fichero limpio
filename = "./Dataset_comentarios_limpios_utf8.csv"
lines_number = 10000  # máximo de filas a leer (limitamos a 10000 para no saturar)

# Carga del CSV limpio en un DataFrame de pandas
data = pd.read_csv(
    filename,
    sep = ";",         # separador del dataset
    encoding = "utf-8",  # todo el contenido en UTF-8
    nrows = lines_number
)

# Eliminamos columnas totalmente vacías
data = data.dropna(axis = 1, how = "all")

print(data.shape)
print(data.columns)
data.head()

(10000, 9)
Index(['MEDIO', 'SOPORTE', 'URL', 'TIPO DE MENSAJE', 'CONTENIDO A ANALIZAR',
       'INTENSIDAD', 'TIPO DE ODIO', 'TONO HUMORISTICO', 'MODIFICADOR'],
      dtype='object')


,MEDIO,SOPORTE,URL,TIPO DE MENSAJE,CONTENIDO A ANALIZAR,INTENSIDAD,TIPO DE ODIO,TONO HUMORISTICO,MODIFICADOR
0,EL PAÍS,WEB,URL_a4d7efc0,COMENTARIO,el barça nunca acaeza ante un segundo b ni ant...,3.0,Otros,NaN,NaN
1,EL PAÍS,WEB,URL_a4d7efc0,COMENTARIO,el real madrid ha puesto punto y final a su an...,0.0,NaN,NaN,NaN
2,EL PAÍS,WEB,URL_54312d9e,COMENTARIO,cristina cifuentes podrí­a haber sido la presi...,3.0,Ideológico,NaN,NaN
3,EL PAÍS,WEB,URL_54312d9e,COMENTARIO,habrí­a que reabrir el caso. el supremo se ded...,3.0,Ideológico,NaN,NaN
4,EL PAÍS,WEB,URL_54312d9e,COMENTARIO,me parece un poco exagerado pedir más de tres ...,3.0,Ideológico,Si,NaN


In [5]:
# Convierto la columna de texto a lista de strings
# Proceso todos los comentarios con spaCy (usando nlp.pipe por eficiencia)
# Creo columnas auxiliares en el DataFrame: DOC, INTENSIDAD, ES_ODIO
# Añado también N_PALABRAS, N_ORACIONES y TIENE_NER

# Convierto la columna de texto a lista de strings, evitando NaN
textos = data["CONTENIDO A ANALIZAR"].fillna("").astype(str).tolist()

doc_list = []
value_list = []

# Procesamiento en batch con nlp.pipe para mejorar el rendimiento
for tmp_doc, tmp_value in zip(nlp.pipe(textos, batch_size = 100), data["INTENSIDAD"]):
    doc_list.append(tmp_doc)    # objeto spaCy Doc
    value_list.append(tmp_value)

# Guardo las listas en variables por si las necesito directamente
doc = doc_list          # lista de documentos spaCy
value = value_list      # intensidades asociadas

# Añado columnas al DataFrame
data["DOC"] = doc
data["INTENSIDAD"] = pd.to_numeric(data["INTENSIDAD"], errors = "coerce")

# ES_ODIO será True si la intensidad es mayor que 0
data["ES_ODIO"] = data["INTENSIDAD"] > 0

print("Número de filas en data:", len(data))
for token in data["DOC"].iloc[0]:
    print(token)

data[["CONTENIDO A ANALIZAR", "INTENSIDAD", "ES_ODIO"]].head()

Número de filas en data: 10000
el
barça
nunca
acaeza
ante
un
segundo
b
ni
ante
un
tercera
,
ya
estan
los
arbitros
para
impedirlo
....
lo
de
messi
es
una
autentica
vergí¼enza


,CONTENIDO A ANALIZAR,INTENSIDAD,ES_ODIO
0,el barça nunca acaeza ante un segundo b ni ant...,3.0,True
1,el real madrid ha puesto punto y final a su an...,0.0,False
2,cristina cifuentes podrí­a haber sido la presi...,3.0,True
3,habrí­a que reabrir el caso. el supremo se ded...,3.0,True
4,me parece un poco exagerado pedir más de tres ...,3.0,True


In [6]:
# En esta celda defino funciones para extraer características de cada comentario a partir del objeto spaCy Doc

def contar_palabras(doc_spacy):
    #Devuelve el número de palabras de contenido en un Doc, excluyendo puntuación y espacios
    return sum(1 for t in doc_spacy if not t.is_punct and not t.is_space)

def contar_oraciones(doc_spacy):
    #Devuelve el número de oraciones en un Doc usando los segmentos de spaCy
    return len(list(doc_spacy.sents))

def tiene_ner(doc_spacy):
    #Indica si el Doc tiene al menos una entidad nombrada
    return len(doc_spacy.ents) > 0

# Aplico las funciones a cada documento y creo columnas derivadas
data["N_PALABRAS"] = data["DOC"].apply(contar_palabras)
data["N_ORACIONES"] = data["DOC"].apply(contar_oraciones)
data["TIENE_NER"] = data["DOC"].apply(tiene_ner)

# Vista de las nuevas columnas
data[["N_PALABRAS", "N_ORACIONES", "TIENE_NER", "ES_ODIO"]].head()

,N_PALABRAS,N_ORACIONES,TIENE_NER,ES_ODIO
0,25,2,True,True
1,195,9,True,False
2,127,7,True,True
3,19,3,False,True
4,147,6,True,True


In [7]:
# Cada fila del DataFrame 'data' corresponde a un comentario. Por tanto, el número de registros es el número de filas
num_registros = len(data)
print(f"El corpus contiene {num_registros} registros")

El corpus contiene 10000 registros


In [10]:
# Uso la columna 'N_PALABRAS', que ya contiene el número de palabras de contenido por comentario y sumo todos los valores
total_palabras = int(data["N_PALABRAS"].sum())
print(f"Total de palabras (tokens) en el corpus: {total_palabras}")

Total de palabras (tokens) en el corpus: 216448


In [11]:
# A partir de 'N_PALABRAS', calculo la media con la función mean() para tener una idea de la longitud típica de los comentarios
promedio_palabras = data["N_PALABRAS"].mean()
print(f"Número promedio de palabras por comentario: {promedio_palabras:.2f}")

Número promedio de palabras por comentario: 21.64


In [12]:
# Palabras por grupo (ODIO vs NO ODIO)
# Número de comentarios por grupo
count_por_grupo = data["ES_ODIO"].value_counts()

# Media de palabras por grupo con la columna N_PALABRAS
medias_palabras = data.groupby("ES_ODIO")["N_PALABRAS"].mean()

print(f"Comentarios con ODIO: {count_por_grupo[True]}")
print(f"Promedio de palabras (ODIO): {medias_palabras[True]:.2f}")
print(f"\nComentarios SIN ODIO: {count_por_grupo[False]}")
print(f"Promedio de palabras (NO ODIO): {medias_palabras[False]:.2f}")

Comentarios con ODIO: 748
Promedio de palabras (ODIO): 22.89

Comentarios SIN ODIO: 9252
Promedio de palabras (NO ODIO): 21.54


In [13]:
# Promedio de oraciones en dos grupos: con odio y sin odio
# Reutilizo 'count_por_grupo' de la celda anterior y calculo la media de oraciones por grupo usando 'N_ORACIONES'
medias_oraciones = data.groupby("ES_ODIO")["N_ORACIONES"].mean()

print(f"Comentarios con ODIO: {count_por_grupo[True]}")
print(f"Promedio de oraciones (ODIO): {medias_oraciones[True]:.2f}")
print(f"\nComentarios SIN ODIO: {count_por_grupo[False]}")
print(f"Promedio de oraciones (NO ODIO): {medias_oraciones[False]:.2f}")

Comentarios con ODIO: 748
Promedio de oraciones (ODIO): 1.76

Comentarios SIN ODIO: 9252
Promedio de oraciones (NO ODIO): 1.78


In [14]:
# Porcentaje de comentarios que contienen entidades NER
# 'sum' indica cuántos tienen entidades y 'count' indica el total de comentarios en cada grupo
tabla_ner = data.groupby("ES_ODIO")["TIENE_NER"].agg(["sum", "count"])

comentarios_con_ner_odio = tabla_ner.loc[True, "sum"]
total_odio = tabla_ner.loc[True, "count"]

comentarios_con_ner_no_odio = tabla_ner.loc[False, "sum"]
total_no_odio = tabla_ner.loc[False, "count"]

porcentaje_ner_odio = comentarios_con_ner_odio / total_odio * 100
porcentaje_ner_no_odio = comentarios_con_ner_no_odio / total_no_odio * 100

print(f"Comentarios con ODIO: {total_odio}")
print(f"Con entidades NER: {comentarios_con_ner_odio} ({porcentaje_ner_odio:.2f}%)")
print(f"\nComentarios SIN ODIO: {total_no_odio}")
print(f"Con entidades NER: {comentarios_con_ner_no_odio} ({porcentaje_ner_no_odio:.2f}%)")

Comentarios con ODIO: 748
Con entidades NER: 286 (38.24%)

Comentarios SIN ODIO: 9252
Con entidades NER: 3119 (33.71%)


In [15]:
# Densidad de signos de puntuación expresivos (!, ?, ...)
import re

puntuacion_odio = 0
puntuacion_no_odio = 0
total_palabras_odio = 0
total_palabras_no_odio = 0

# Recorro todos los comentarios y cuento signos del tipo: !, ?, !!, ??, ..., etc.
for i in range(len(data)):
    texto = data["CONTENIDO A ANALIZAR"].iloc[i]
    if isinstance(texto, str):
        # Capturo secuencias de !, ?, o varios puntos
        signos_expresivos = len(re.findall(r'[!?]+|\.{2,}', texto))
        num_palabras = data["N_PALABRAS"].iloc[i]

        if data["ES_ODIO"].iloc[i]:
            puntuacion_odio += signos_expresivos
            total_palabras_odio += num_palabras
        else:
            puntuacion_no_odio += signos_expresivos
            total_palabras_no_odio += num_palabras

# Normalizo por número total de palabras del grupo y lo convierto a porcentaje
densidad_odio = (puntuacion_odio / total_palabras_odio * 100) if total_palabras_odio > 0 else 0
densidad_no_odio = (puntuacion_no_odio / total_palabras_no_odio * 100) if total_palabras_no_odio > 0 else 0

print(f"ODIO - Densidad de puntuación expresiva: {densidad_odio:.2f}%")
print(f"NO ODIO - Densidad de puntuación expresiva: {densidad_no_odio:.2f}%")

ODIO - Densidad de puntuación expresiva: 2.59%
NO ODIO - Densidad de puntuación expresiva: 2.58%


In [17]:
#Proporción de palabras ofensivas como un pequeño lexicón de insultos 
#Trabajo con lemas en minúsculas para obtener variaciones morfológicas
lexico_ofensivo = {
    "idiota", "imbécil", "imbecil", "asqueroso", "asquerosa",
    "mierda", "mierdas", "payaso", "payasa", "cerdo", "cerda",
    "estúpido", "estupido", "estúpida", "tonto", "tonta",
    "gilipollas", "subnormal", "basura", "rata", "escoria"
}

ofensivas_odio = 0
ofensivas_no_odio = 0
total_palabras_odio = 0
total_palabras_no_odio = 0

# Recorro cada documento spaCy y miro los lemas de los tokens alfabéticos
for i, doc_i in enumerate(doc):
    for token in doc_i:
        if token.is_alpha:
            lema = token.lemma_.lower()
            if data["ES_ODIO"].iloc[i]:
                total_palabras_odio += 1
                if lema in lexico_ofensivo:
                    ofensivas_odio += 1
            else:
                total_palabras_no_odio += 1
                if lema in lexico_ofensivo:
                    ofensivas_no_odio += 1

prop_ofensivas_odio = (ofensivas_odio / total_palabras_odio * 100) if total_palabras_odio > 0 else 0
prop_ofensivas_no_odio = (ofensivas_no_odio / total_palabras_no_odio * 100) if total_palabras_no_odio > 0 else 0

print(f"ODIO - Palabras ofensivas: {ofensivas_odio} ({prop_ofensivas_odio:.4f}%)")
print(f"NO ODIO - Palabras ofensivas: {ofensivas_no_odio} ({prop_ofensivas_no_odio:.4f}%)")

ODIO - Palabras ofensivas: 140 (0.8717%)
NO ODIO - Palabras ofensivas: 352 (0.1879%)


In [18]:
# Diversidad léxica (Type-Token Ratio, TTR)
tokens_unicos_odio = set()
tokens_unicos_no_odio = set()
total_tokens_odio = 0
total_tokens_no_odio = 0

# Recorro todos los documentos y recojo las palabras alfabéticas en minúsculas, diferenciando odio y sin odio
for i in range(len(doc)):
    for token in doc[i]:
        if token.is_alpha:
            palabra_lower = token.text.lower()
            
            if data["ES_ODIO"].iloc[i]:
                total_tokens_odio += 1
                tokens_unicos_odio.add(palabra_lower)
            else:
                total_tokens_no_odio += 1
                tokens_unicos_no_odio.add(palabra_lower)

ttr_odio = (len(tokens_unicos_odio) / total_tokens_odio * 100) if total_tokens_odio > 0 else 0
ttr_no_odio = (len(tokens_unicos_no_odio) / total_tokens_no_odio * 100) if total_tokens_no_odio > 0 else 0

print(f"ODIO - Palabras únicas: {len(tokens_unicos_odio)} de {total_tokens_odio}")
print(f"ODIO - Type-Token Ratio (TTR): {ttr_odio:.2f}%")
print(f"\nNO ODIO - Palabras únicas: {len(tokens_unicos_no_odio)} de {total_tokens_no_odio}")
print(f"NO ODIO - Type-Token Ratio (TTR): {ttr_no_odio:.2f}%")

ODIO - Palabras únicas: 4448 de 16061
ODIO - Type-Token Ratio (TTR): 27.69%

NO ODIO - Palabras únicas: 22935 de 187299
NO ODIO - Type-Token Ratio (TTR): 12.25%


In [19]:
# Proporción de verbos y conteo de verbos en modo imperativo
verbos_odio = 0
verbos_no_odio = 0
total_palabras_odio = 0
total_palabras_no_odio = 0

# Cuento cuántos tokens son VERB y cuántas palabras alfabéticas hay por grupo
for i in range(len(doc)):
    for token in doc[i]:
        if token.is_alpha:
            if data["ES_ODIO"].iloc[i]:
                total_palabras_odio += 1
                if token.pos_ == "VERB":
                    verbos_odio += 1
            else:
                total_palabras_no_odio += 1
                if token.pos_ == "VERB":
                    verbos_no_odio += 1

prop_verbos_odio = (verbos_odio / total_palabras_odio * 100) if total_palabras_odio > 0 else 0
prop_verbos_no_odio = (verbos_no_odio / total_palabras_no_odio * 100) if total_palabras_no_odio > 0 else 0

print(f"ODIO - Verbos: {verbos_odio} ({prop_verbos_odio:.2f}%)")
print(f"NO ODIO - Verbos: {verbos_no_odio} ({prop_verbos_no_odio:.2f}%)")

# Verbos en modo imperativo según la morfología de spaCy
verbos_imp_odio = 0
verbos_imp_no_odio = 0

for i in range(len(doc)):
    for token in doc[i]:
        if token.pos_ == "VERB":
            if "Mood=Imp" in token.morph:
                if data["ES_ODIO"].iloc[i]:
                    verbos_imp_odio += 1
                else:
                    verbos_imp_no_odio += 1

print("Verbos imperativos en ODIO:", verbos_imp_odio)
print("Verbos imperativos en NO ODIO:", verbos_imp_no_odio)

ODIO - Verbos: 1824 (11.36%)
NO ODIO - Verbos: 23497 (12.55%)
Verbos imperativos en ODIO: 31
Verbos imperativos en NO ODIO: 414


In [20]:
# Análisis comparativo de todas las características
import numpy as np

print("RESUMEN COMPARATIVO DE CARACTERÍSTICAS")

# Porcentaje de comentarios con NER por grupo
porcentaje_ner = data.groupby("ES_ODIO")["TIENE_NER"].mean() * 100
print("-"*80)

# Diccionario con las características calculadas
caracteristicas = {
    "Promedio de palabras": (medias_palabras[True], medias_palabras[False]),
    "Promedio de oraciones": (medias_oraciones[True], medias_oraciones[False]),
    "% con entidades NER": (porcentaje_ner[True], porcentaje_ner[False]),
    "Densidad puntuación expresiva (%)": (densidad_odio, densidad_no_odio),
    "% Palabras ofensivas (lexicón simple)": (prop_ofensivas_odio, prop_ofensivas_no_odio),
    "Type-Token Ratio (%)": (ttr_odio, ttr_no_odio),
    "% Verbos": (prop_verbos_odio, prop_verbos_no_odio)
}

print(f"\n{'Característica':<40} {'ODIO':>10} {'NO ODIO':>10} {'Diferencia':>12}")
print("-"*80)

# Imprimo para cada característica los valores en cada grupo y la diferencia relativa
for nombre, (odio, no_odio) in caracteristicas.items():
    diferencia = abs(odio - no_odio)
    diff_relativa = (diferencia / no_odio * 100) if no_odio != 0 else 0
    print(f"{nombre:<40} {odio:>10.2f} {no_odio:>10.2f} {diff_relativa:>11.2f}%")

print("\n" + "-"*80)
print("CONCLUSIÓN:")
print("-"*80)
print("""
La característica con MAYOR PODER DISCRIMINATORIO es:
- Type-Token Ratio (TTR) - Diversidad léxica

- Los mensajes de odio tienden a mostrar una mayor diversidad léxica
- Esta característica podría usarse efectivamente como entrada de un clasificador

La característica de porcentaje de palabras ofensivas también resulta muy relevante:
- Se basa en un lexicón explícito de insultos
- Captura carga lingüística agresiva directamente relacionada con el discurso de odio

Características con poder discriminatorio MODERADO:
- Promedio de palabras por comentario
- % de comentarios con entidades NER

Características con poco o nulo poder discriminatorio:
- Promedio de oraciones
- Densidad de puntuación expresiva
- % de verbos sobre el total de palabras
""")

RESUMEN COMPARATIVO DE CARACTERÍSTICAS
--------------------------------------------------------------------------------

Característica                                 ODIO    NO ODIO   Diferencia
--------------------------------------------------------------------------------
Promedio de palabras                          22.89      21.54        6.27%
Promedio de oraciones                          1.76       1.78        0.81%
% con entidades NER                           38.24      33.71       13.42%
Densidad puntuación expresiva (%)              2.59       2.58        0.14%
% Palabras ofensivas (lexicón simple)          0.87       0.19      363.82%
Type-Token Ratio (%)                          27.69      12.25      126.17%
% Verbos                                      11.36      12.55        9.47%

--------------------------------------------------------------------------------
CONCLUSIÓN:
--------------------------------------------------------------------------------

La característ